In [1]:
# Importando as bibliotecas

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import IsolationForest

In [2]:
# Para remover os warnings
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [3]:
# Carregando a base de dados

fraud = pd.read_csv("creditcard.csv", sep = ",")
df = fraud.copy()

fraud.shape[0]

284807

In [4]:
# Visualizando a tabela dos dados
fraud.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
# Estatisticas da base

fraud.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,3.919560e-15,5.688174e-16,-8.769071e-15,2.782312e-15,-1.552563e-15,2.010663e-15,-1.694249e-15,-1.927028e-16,-3.137024e-15,...,1.537294e-16,7.959909e-16,5.367590e-16,4.458112e-15,1.453003e-15,1.699104e-15,-3.660161e-16,-1.206049e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [6]:
# Existem missing values?

fraud.isnull().values.any()

False

In [7]:
# Distribuiçao entre as classes?

print("Percentual de fraudes: {}".format(round(fraud[fraud["Class"] == 1].shape[0]/fraud.shape[0]*100, 3)))
print("Percentual de não fraudes: {}".format(round(fraud[fraud["Class"] == 0].shape[0]/fraud.shape[0]*100, 3)))

Percentual de fraudes: 0.173
Percentual de não fraudes: 99.827


In [8]:
# Estatisticas para fraudes (1) e nao fraudes (0)

fraud_df = pd.DataFrame(fraud[fraud["Class"] == 1]["Amount"].describe())
fraud_df.columns = ['Valores de Fraude']
fraud_df["Valores sem Fraude"] = pd.Series(fraud[fraud["Class"] == 0]["Amount"].describe())
fraud_df

,Valores de Fraude,Valores sem Fraude
count,492.000000,284315.000000
mean,122.211321,88.291022
std,256.683288,250.105092
min,0.000000,0.000000
25%,1.000000,5.650000
50%,9.250000,22.000000
75%,105.890000,77.050000
max,2125.870000,25691.160000


In [9]:

# Separando os dados com as variáveis em x e o alvo em y
x = fraud.drop("Class", axis = 1)
y = fraud["Class"]

# Separando entre treinamento e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.5, random_state = 42)

In [10]:
x

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,1.475829,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.059616,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.001396,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.127434,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00


In [11]:
# Treinando o modelo
iForest = IsolationForest(n_estimators = 200, max_samples = len(x), random_state = 42) #Informando parâmetros do modelo

iForest.fit(x_train) #Comando que treina o Modelo

IsolationForest(max_samples=284807, n_estimators=200, random_state=42)

In [12]:
# Predições para o treinamento e teste

y_pred_train1 = iForest.predict(x_train)
y_pred_test1 = iForest.predict(x_test)

In [13]:
y_pred_test1

array([-1,  1,  1, ...,  1,  1,  1])

In [16]:
# Reajustando a previsão para ficar alinhado com a Class da base

y_pred_train1 = np.where(y_pred_train1 == -1, 1, 0)
y_pred_test1 = np.where(y_pred_test1 == -1, 1, 0)

In [17]:
print(np.count_nonzero(y_pred_train1 == 1))

print(np.count_nonzero(y_train == 1))

331
246


In [18]:
print(np.count_nonzero(y_pred_test1 == 1))

print(np.count_nonzero(y_test == 1))

320
246


In [19]:
print("Base de treinamento")
print("Percentual de fraudes: {}".format(round(accuracy_score(y_train,y_pred_train1)*100, 2)))
print("------------------------------")
print("Base de teste")
print("Percentual de fraudes: {}".format(round(accuracy_score(y_test,y_pred_test1)*100, 2)))

Base de treinamento
Percentual de fraudes: 99.71
------------------------------
Base de teste
Percentual de fraudes: 99.74


In [20]:
# Treinamento

print(confusion_matrix(y_train, y_pred_train1))
print()
print("--------------------------------------------------------------------------------")
print()
print(classification_report(y_train, y_pred_train1))

[[141906    251]
 [   166     80]]

--------------------------------------------------------------------------------

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    142157
           1       0.24      0.33      0.28       246

    accuracy                           1.00    142403
   macro avg       0.62      0.66      0.64    142403
weighted avg       1.00      1.00      1.00    142403



In [21]:
# Teste

print(confusion_matrix(y_test, y_pred_test1))
print()
print("--------------------------------------------------------------------------------")
print()
print(classification_report(y_test, y_pred_test1))

[[141938    220]
 [   146    100]]

--------------------------------------------------------------------------------

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    142158
           1       0.31      0.41      0.35       246

    accuracy                           1.00    142404
   macro avg       0.66      0.70      0.68    142404
weighted avg       1.00      1.00      1.00    142404



In [22]:

# Essa biblioteca coloca o número já na moeda local, sem que tenhamos que ficar preenchendo a mão
# de acordo com a localização do seu computador

import locale
locale.setlocale( locale.LC_ALL, '' )

'pt_BR.UTF-8'

In [23]:
print("Transação Normal")
print("Valor médio: {}".format(locale.currency(round(fraud[fraud["Class"] == 0]["Amount"].describe()[1], 2))))
print("___________________________")
print("Transação Fraudulenta")
print("Valor médio: {}".format(locale.currency(round(fraud[fraud["Class"] == 1]["Amount"].describe()[1], 2))))

Transação Normal
Valor médio: R$ 88,29
___________________________
Transação Fraudulenta
Valor médio: R$ 122,21


In [24]:
print("Teriamos bloqueado mediamente: {}".format(locale.currency(
    round(fraud[fraud["Class"] == 1]["Amount"].describe()[1]*99, 2), grouping=True)))
print("------------------------------")
print("Ao custo médio de : {}".format(locale.currency(
    round(fraud[fraud["Class"] == 0]["Amount"].describe()[1]*215, 2), grouping=True)))

Teriamos bloqueado mediamente: R$ 12.098,92
------------------------------
Ao custo médio de : R$ 18.982,57


In [25]:
# Vamos olhar o valor real para a base inteira

# aqui usei o mesmo dataset mas como nome de "df", simplesmente para não impactar o dataset original.

df["IF"] = np.where(iForest.predict(df.iloc[:, 0:30]) == -1, 1, 0)

In [26]:
locale.currency(df[df["Class"] == 1]["Amount"].sum())

'R$ 60127,97'

In [27]:
locale.currency(df[df["IF"] == 1]["Amount"].sum())

'R$ 831237,37'

In [28]:
print("Teríamos bloqueado: {}".format(locale.currency(df[(df["IF"] == 1) & (df["Class"] == 1)]["Amount"].sum()
                                                      , grouping=True)))
print()
print("Que representa % do total: {}".format(round(df[(df["IF"] == 1) & (df["Class"] == 1)]["Amount"].sum()/
                                              df[df["Class"] == 1]["Amount"].sum()*100, 2)))
print()
print("Autorizando indevidamente: {}".format(locale.currency(df[(df["IF"] == 0) & (fraud["Class"] == 1)]["Amount"].sum()
                                                    , grouping=True)))
print("------------------------------")
print("Ao custo de: {}".format(locale.currency(df[(df["IF"] == 1) & (df["Class"] == 0)]["Amount"].sum(), grouping=True)))
print("")
print("Que representa % do total: {}".format(round(df[(df["IF"] == 0) & (df["Class"] == 0)]["Amount"].sum()/
                                              df[df["Class"] == 0]["Amount"].sum()*100, 2)))

Teríamos bloqueado: R$ 15.454,34

Que representa % do total: 25.7

Autorizando indevidamente: R$ 44.673,63
------------------------------
Ao custo de: R$ 815.783,03

Que representa % do total: 96.75


In [29]:

# Separando os dados com as variáveis em x e o alvo em y
x = df.drop(["Class","IF"], axis = 1)
y = df["Class"]

# Separando entre treinamento e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.5, random_state = 42)

In [30]:
# Treinando o modelo

iForest2 = IsolationForest(n_estimators = 1, max_samples = 100000, random_state = 42)

iForest2.fit(x_train)

IsolationForest(max_samples=100000, n_estimators=1, random_state=42)

In [31]:
# Prediçoes para o treinamento e teste

y_pred_train2 = iForest2.predict(x_train)
y_pred_test2 = iForest2.predict(x_test)

# Returns -1 for outliers and 1 for inliers.

In [32]:
# Returns -1 for outliers and 1 for inliers.

# Reajustando a previsao para ficar alinhado com a Class da base

y_pred_train2 = np.where(y_pred_train2 == -1, 1, 0)
y_pred_test2 = np.where(y_pred_test2 == -1, 1, 0)

In [33]:
print("Base de treinamento")
print("Percentual de fraudes: {}".format(round(accuracy_score(y_train,y_pred_train2)*100, 2)))
print("------------------------------")
print("Base de teste")
print("Percentual de fraudes: {}".format(round(accuracy_score(y_test,y_pred_test2)*100, 2)))

Base de treinamento
Percentual de fraudes: 99.48
------------------------------
Base de teste
Percentual de fraudes: 99.48


In [34]:
# Mas qual o valor real?

df["IF2"] = np.where(iForest2.predict(df.iloc[:, 0:30]) == -1, 1, 0)

In [35]:
print("Teriamos bloqueado: {}".format(locale.currency(df[(df["IF2"] == 1) & (df["Class"] == 1)]["Amount"].sum()
                                                      , grouping=True)))
print()
print("% do total: {}".format(round(df[(df["IF2"] == 1) & (df["Class"] == 1)]["Amount"].sum()/
                                              df[df["Class"] == 1]["Amount"].sum()*100, 2)))
print()
print("Deixando passar : {}".format(locale.currency(df[(df["IF2"] == 0) & (df["Class"] == 1)]["Amount"].sum()
                                                    , grouping=True)))
print("------------------------------")
print("Ao custo de : {}".format(locale.currency(df[(df["IF2"] == 1) & (df["Class"] == 0)]["Amount"].sum(), grouping=True)))
print("")
print("% do total ainda aceito: {}".format(round(df[(df["IF2"] == 0) & (df["Class"] == 0)]["Amount"].sum()/
                                              df[df["Class"] == 0]["Amount"].sum()*100, 2)))

Teriamos bloqueado: R$ 16.076,07

% do total: 26.74

Deixando passar : R$ 44.051,90
------------------------------
Ao custo de : R$ 681.367,22

% do total ainda aceito: 97.29
